import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=2
btch=20
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ProximalPhalanxOutlineCorrect(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (600, 81)
first_test_shape= (291, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [406]
max(train_feature_Altitude)= 1.9029
min(train_feature_Altitude)= -1.4834
first_train_sample=
 [ 1.       -0.70673  -0.65214  -0.45675  -0.21008   0.075267  0.34994
  0.59501   0.77936   1.0623    1.3086    1.4932    1.5211    1.5118
  1.3693    1.169     0.9772    0.71438   0.49046   0.23282  -0.054244
 -0.31903  -0.57195  -0.72026  -0.72978  -0.6763   -0.4762   -0.22279
  0.044653  0.33074   0.62671   0.90485   1.1938    1.4955    1.6582
  1.6582    1.5609    1.5457    1.5377    1.5915    1.6089    1.6576
  1.4566    1.1623    0.86912   0.59661   0.29749   0.034627 -0.21209
 -0.47783  -0.65511  -0.57193  -0.65446  -0.75273  -0.92547  -1.0641
 -1.218    -1.33     -1.3396   -1.2731   -1.1312   -0.98085  -0.84127
 -0.74757  -0.64479  -0.62498  -0.6119   -0.70658  -0.7906   -0.9123
 -1.0589   -1.1936   -1.3203   -1.391    -1.3003   -1.1547   -1.0317
 -0.8765   -0.75764  -0.607

In [3]:
np.shape(ecg)

(600, 481)

In [4]:
ecg[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 194	1 406	
 max = 406


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (194, 481)
Up to class  1 train shape =  (600, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[194 406]


In [8]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  1.7743120350766923
min magnitude class 0  =  -1.299820826698177
after normalizing >>
max magnitude class 0  =  0.9194507121403934
min magnitude class 0  =  -0.8879297726404382

 cls 1  >> 
max magnitude class 1  =  1.8237000000000003
min magnitude class 1  =  -1.4568201800265863
after normalizing >>
max magnitude class 1  =  0.9484874675439612
min magnitude class 1  =  -0.9802346859567573
each class and its windows =
0 92	1 199	
 max instance in classes of test windows = 199


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9802346859567573
max tst = 0.9484874675439612


In [14]:
len(wndws_test)

291

In [15]:
np.shape(wndws_test)

(291, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(406, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (600, 480)
y_train =>  (600,)
X_test  =>  (291, 480)
y_test  =>  (291,)
X_valid  =>  (600, 480)
y_valid  =>  (600,)


In [22]:
if min(y_train)==1:
    y_train=y_train-1
    y_test=y_test-1
    y_valid =y_valid-1
print('Class labels=',set(y_train))

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 0, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
8/8 [==============================] - 10s 109ms/step - loss: 2.7824 - accuracy: 0.4833 - val_loss: 0.9851 - val_accuracy: 0.6767
Epoch 2/1000
8/8 [==============================] - 2s 313ms/step - loss: 0.6423 - accuracy: 0.6767 - val_loss: 0.9375 - val_accuracy: 0.6767
Epoch 3/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.6663 - accuracy: 0.5933 - val_loss: 0.8874 - val_accuracy: 0.6767
Epoch 4/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.6190 - accuracy: 0.6217 - val_loss: 0.8300 - val_accuracy: 0.6767
Epoch 5/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.6302 - accuracy: 0.6700 - val_loss: 0.7962 - val_accuracy: 0.6767
Epoch 6/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.5843 - accuracy: 0.6950 - val_loss: 0.7789 - val_accuracy: 0.6767
Epoch 7/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.5548 - accuracy: 0.6667 - val_loss: 0.7543 - val_accuracy: 0.6767
Epoch 8/10

Epoch 59/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.2992 - accuracy: 0.8700 - val_loss: 0.3257 - val_accuracy: 0.8800
Epoch 60/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.2598 - accuracy: 0.8933 - val_loss: 0.4519 - val_accuracy: 0.8217
Epoch 61/1000
8/8 [==============================] - 0s 25ms/step - loss: 0.2572 - accuracy: 0.8900 - val_loss: 0.2423 - val_accuracy: 0.9200
Epoch 62/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.2624 - accuracy: 0.8983 - val_loss: 0.3796 - val_accuracy: 0.9017
Epoch 63/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.2557 - accuracy: 0.9083 - val_loss: 0.4162 - val_accuracy: 0.8433
Epoch 64/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.3094 - accuracy: 0.8950 - val_loss: 0.2387 - val_accuracy: 0.9133
Epoch 65/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.2041 - accuracy: 0.9133 - val_loss: 0.2789 - val_accuracy: 0.8667
Epoch 

8/8 [==============================] - 0s 24ms/step - loss: 0.0574 - accuracy: 0.9850 - val_loss: 0.2080 - val_accuracy: 0.9300
Epoch 174/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.1037 - accuracy: 0.9567 - val_loss: 0.0833 - val_accuracy: 0.9700
Epoch 175/1000
8/8 [==============================] - 0s 25ms/step - loss: 0.1156 - accuracy: 0.9750 - val_loss: 0.0690 - val_accuracy: 0.9667
Epoch 176/1000
8/8 [==============================] - 0s 23ms/step - loss: 0.0485 - accuracy: 0.9850 - val_loss: 0.0236 - val_accuracy: 0.9917
Epoch 177/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.0572 - accuracy: 0.9767 - val_loss: 0.1194 - val_accuracy: 0.9683
Epoch 178/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.0755 - accuracy: 0.9800 - val_loss: 0.0927 - val_accuracy: 0.9700
Epoch 179/1000
8/8 [==============================] - 0s 24ms/step - loss: 0.0290 - accuracy: 0.9867 - val_loss: 0.0633 - val_accuracy: 0.9800
Epoch 180/1000

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 15ms/step
array([[ 76,  16],
       [  6, 193]], dtype=int64)
              precision    recall  f1-score   support

           1       0.93      0.83      0.87        92
           2       0.92      0.97      0.95       199

    accuracy                           0.92       291
   macro avg       0.93      0.90      0.91       291
weighted avg       0.92      0.92      0.92       291



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 0, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
8/8 [==============================] - 2s 112ms/step - loss: 13.2013 - accuracy: 0.3850 - val_loss: 0.9456 - val_accuracy: 0.6767
Epoch 2/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.6199 - accuracy: 0.6767 - val_loss: 0.9519 - val_accuracy: 0.6767
Epoch 3/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.5996 - accuracy: 0.6750 - val_loss: 0.8748 - val_accuracy: 0.6767
Epoch 4/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.6574 - accuracy: 0.6300 - val_loss: 0.8792 - val_accuracy: 0.6767
Epoch 5/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.6404 - accuracy: 0.6567 - val_loss: 0.8508 - val_accuracy: 0.3233
Epoch 6/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.6628 - accuracy: 0.6050 - val_loss: 0.7598 - val_accuracy: 0.6767
Epoch 7/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.6756 - accuracy: 0.6083 - val_loss: 0.8283 - val_accuracy: 0.6767
Epoch 8/100

Epoch 59/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.3657 - accuracy: 0.8300 - val_loss: 0.6872 - val_accuracy: 0.6167
Epoch 60/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.4213 - accuracy: 0.7950 - val_loss: 0.4239 - val_accuracy: 0.8317
Epoch 61/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.3964 - accuracy: 0.8217 - val_loss: 0.4488 - val_accuracy: 0.8183
Epoch 62/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.3580 - accuracy: 0.8350 - val_loss: 1.8731 - val_accuracy: 0.7483
Epoch 63/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.5425 - accuracy: 0.8200 - val_loss: 0.4221 - val_accuracy: 0.7867
Epoch 64/1000
8/8 [==============================] - 0s 37ms/step - loss: 0.3508 - accuracy: 0.8467 - val_loss: 0.3691 - val_accuracy: 0.8567
Epoch 65/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.3989 - accuracy: 0.8183 - val_loss: 0.3661 - val_accuracy: 0.8400
Epoch 

8/8 [==============================] - 0s 36ms/step - loss: 0.1262 - accuracy: 0.9633 - val_loss: 0.0533 - val_accuracy: 0.9767
Epoch 174/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.1054 - accuracy: 0.9567 - val_loss: 0.1231 - val_accuracy: 0.9517
Epoch 175/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.0713 - accuracy: 0.9650 - val_loss: 0.1233 - val_accuracy: 0.9617
Epoch 176/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.2674 - accuracy: 0.9000 - val_loss: 0.1199 - val_accuracy: 0.9517
Epoch 177/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.1416 - accuracy: 0.9400 - val_loss: 0.1149 - val_accuracy: 0.9567
Epoch 178/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.0738 - accuracy: 0.9733 - val_loss: 0.0600 - val_accuracy: 0.9733
Epoch 179/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.1010 - accuracy: 0.9600 - val_loss: 0.0624 - val_accuracy: 0.9717
Epoch 180/1000

8/8 [==============================] - 0s 34ms/step - loss: 0.0269 - accuracy: 0.9900 - val_loss: 0.0228 - val_accuracy: 0.9900
Epoch 288/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.0142 - accuracy: 0.9933 - val_loss: 0.0193 - val_accuracy: 0.9933
Epoch 289/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.1034 - accuracy: 0.9783 - val_loss: 0.1407 - val_accuracy: 0.9567
Epoch 290/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.3536 - accuracy: 0.9250 - val_loss: 0.1294 - val_accuracy: 0.9583
Epoch 291/1000
8/8 [==============================] - 0s 34ms/step - loss: 0.0368 - accuracy: 0.9833 - val_loss: 0.0486 - val_accuracy: 0.9783
Epoch 292/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.0120 - accuracy: 0.9983 - val_loss: 0.1029 - val_accuracy: 0.9717
Epoch 293/1000
8/8 [==============================] - 0s 35ms/step - loss: 0.0712 - accuracy: 0.9800 - val_loss: 0.1717 - val_accuracy: 0.9383
Epoch 294/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 15ms/step
array([[ 73,  19],
       [  5, 194]], dtype=int64)
              precision    recall  f1-score   support

           1       0.94      0.79      0.86        92
           2       0.91      0.97      0.94       199

    accuracy                           0.92       291
   macro avg       0.92      0.88      0.90       291
weighted avg       0.92      0.92      0.92       291

